Name: run_wc_dataset;

Version: 2.0;

Author: Anish Vasan;

Organization: Chen/Eyckmans Lab, Boston University;

Date: 20230403;

## IMPORTS ##

In [34]:
import os, shutil, fnmatch, tkinter, yaml
from tkinter.filedialog import askdirectory
from pathlib import Path
from woundcompute import image_analysis as ia
import time

## DEFINITIONS ##
## These require user input/changes based on experiments ##

In [35]:
image_type = "ph1"  #Options:  bf, ph1
is_fl = False #Options: True, False
is_track = False #Options: True, False

## FUNCTIONS ##

In [36]:
def create_yaml(path: str, image_type_in: str):
    """Given the output path as string. Will create a yaml file in the main output folder. This yaml file will be copied into each subfolder during the sorting function"""

    #Default keys and values stored in yaml file
    yaml_input_file = {
    'version' : 1.0,
    'segment_brightfield' : False,
    'seg_bf_version' : 1,
    'seg_bf_visualize' : False,
    'segment_fluorescent' : False,
    'seg_fl_version' : 1,
    'seg_fl_visualize' : False,
    'segment_ph1' : False, #True,
    'seg_ph1_version' : 2,
    'seg_ph1_visualize' : False, #True,
    'track_brightfield' : False,
    'track_bf_version' : 1,
    'track_bf_visualize' : False,
    'track_ph1' : False,
    'track_ph1_version' : 1,
    'track_ph1_visualize' : False,
    'bf_seg_with_fl_seg_visualize' : False,
    'bf_track_with_fl_seg_visualize' : False,
    'ph1_seg_with_fl_seg_visualize' : False,
    'ph1_track_with_fl_seg_visualize' : False,
    }

    #Conditionally modify yaml file based on image_type input

    for key, value in yaml_input_file.items():
        if image_type in key and not "version" in key and not "fl" in key and not "track" in key:
            yaml_input_file[key] = True

            if is_fl:
                if "fl" in key and not "version" in key:
                    yaml_input_file[key] = True

            if is_track:
                if "track" in key and not "version" in key:
                    yaml_input_file[key] = True

    # Write yaml output to path_output
    with open(path+'/wc_dataset_'+image_type_in+'.yaml', 'w') as file:
        yaml.safe_dump(yaml_input_file, file, sort_keys=False)



In [37]:
def define_basename_list(path_input_fn: str) -> list:
    """Given an input path as a string. Will return a list of experiment names in the input folder"""

    basename_list_fn = []
    for file in os.listdir(path_input_fn):
        if file.endswith('.nd'):
            name, ext = os.path.splitext(file)
            basename_list_fn.append(name)
    return basename_list_fn

In [38]:
def sort_basename_folders(basename_list_fn: list, path_input_fn: str, path_output_fn: str):
    """Given a list of experiments obtained from the .nd files in the input folder. Given input and out paths as str. Creates an output folder at path_output. Copies and sorts TIFF files in path_input according to basenames in output folder """

    for basename_fn in basename_list_fn:

        #create folders for each expt
        os.makedirs(path_output_fn + '/' + basename_fn)

        #copy .nd file into respective basename folder
        shutil.copy(path_input_fn + '/' + basename_fn + '.nd', path_output_fn + '/' + basename_fn + '/' + basename_fn + '.nd')

        #copy TIF files to respective basename folders, excludes thumbnail files
        for file in os.listdir(path_input_fn):
            if file.startswith(basename_fn + '_') and not fnmatch.fnmatch(file, '*thumb*'):
                shutil.copy(path_input_fn + '/' + file, path_output_fn + '/' + basename_fn + '/' + file)

In [39]:
def extract_nd_info(basename_list_fn: list,path_output_fn: str)-> list:
    """Given a basename list and an output path. Will extract number of stage positions for each list and create position map lists for each basename"""

    stage_positions = []
    stage_pos_maps = []
    for index_fn, basename_fn in basename_list_fn:
        with open(path_output_fn + '/' + basename_fn + '/' + basename_fn + '.nd', 'r') as nd_file:
            for l_no, line in enumerate(nd_file):
                if '"NStagePositions"' in line:
                    stage_positions[index_fn].append([int(s) for s in line.split() if s.isdigit()][-1])
                    break


    return stage_positions

In [48]:
def sort_stage_pos_folders(basename_list_fn: list,parent_output_fn: str, stage_pos_nd_fn: list, image_type_fn: str):
    """Given basename list, parent output folder and number of stage positions list extracted from nd files. Sorts files into stage position folders"""
    #sort images into stage position folders
    for index_fn,basename_fn in enumerate(basename_list_fn):

        for file in os.listdir(parent_output_fn+'/'+basename_fn):

            for stage_pos_fn in range(1,stage_pos_nd_fn[index_fn]+1):

                #define input and output paths
                path_input_fn  = parent_output_fn + '/' + basename_fn
                path_pos_output_fn =  parent_output_fn + '/' + basename_fn + '/' + 's' + f"{stage_pos_fn:02}"


                #check if file belongs to current stage_pos group
                if fnmatch.fnmatch(file, '*s%i' % stage_pos_fn+'_*'):

                #rename file for correct stage position sort syntax (_sXX_)

                    #Isolate the frame number (timepoint)
                    file_timepoint = file.split('_t', 1)[-1].split('.', 1)[0]
                    # Rename file for correct time and stage position sort syntax (_sXX_ and _tXXX.)
                    file_newname = 's' + f"{stage_pos_fn:02}" +'_t'+f"{int(file_timepoint):03}"+'.TIF'

                    #check if stage position directory already exists and move file into this folder if it does
                    if os.path.exists(path_pos_output_fn + '/' + image_type_fn+ '_images/'):
                        try:
                            shutil.move(path_input_fn + '/' + file, path_pos_output_fn + '/' + image_type_fn+ '_images/'+ file_newname)
                        except OSError as e:
                            # If it fails, inform the user.
                            print('Error: %s - %s.' % (e.filename, e.strerror))
                    #otherwise, create folder and move file into this folder. Also copy parent yaml file into this folder.
                    else:
                        try:
                            os.makedirs(path_pos_output_fn + '/' + image_type_fn+ '_images/')
                            shutil.copy(parent_output_fn + '/wc_dataset_'+image_type_fn+'.yaml', path_pos_output_fn + '/wc_dataset_'+image_type_fn+'.yaml')
                            shutil.move(path_input_fn + '/' + file, path_pos_output_fn + '/' + image_type_fn+ '_images/'+ file_newname)


                        except OSError as e:
                            # If it fails, inform the user.
                            print('Error: %s - %s.' % (e.filename, e.strerror))

                elif fnmatch.fnmatch(file, '*s%i' % stage_pos_fn+'.TIF'):

                    #If data does not have timepoints, rename stage positions only for correct sort syntax
                    file_newname = 's' + f"{stage_pos_fn:02}" + '.TIF'

                    #check if stage position directory already exists and move file into this folder if it does
                    if os.path.exists(path_pos_output_fn + '/' + image_type_fn+ '_images/'):
                        try:
                            shutil.move(path_input_fn + '/' + file, path_pos_output_fn + '/' + image_type_fn+ '_images/'+ file_newname)
                        except OSError as e:
                            # If it fails, inform the user.
                            print('Error: %s - %s.' % (e.filename, e.strerror))
                    #otherwise, create folder and move file into this folder. Also copy parent yaml file into this folder.
                    else:
                        try:
                            os.makedirs(path_pos_output_fn + '/' + image_type_fn+ '_images/')
                            shutil.copy(parent_output_fn + '/wc_dataset_'+image_type_fn+'.yaml', path_pos_output_fn + '/wc_dataset_'+image_type_fn+'.yaml')
                            shutil.move(path_input_fn + '/' + file, path_pos_output_fn + '/' + image_type_fn+ '_images/'+ file_newname)


                        except OSError as e:
                            # If it fails, inform the user.
                            print('Error: %s - %s.' % (e.filename, e.strerror))


In [59]:
basename_list_fn = ['tissue_ai']
parent_output_fn = path_output
stage_pos_nd_fn= [13]
image_type_fn = "ph1"
for index_fn,basename_fn in enumerate(basename_list_fn):

        for file in os.listdir(parent_output_fn+'/'+basename_fn):
            for stage_pos_fn in range(1,stage_pos_nd_fn[index_fn]+1):

                #check if file belongs to current stage_pos group
                if fnmatch.fnmatch(file, '*s%i' % stage_pos_fn+'_*'):
                    #define input and output paths
                    path_input_fn  = parent_output_fn + '/' + basename_fn
                    path_pos_output_fn =  parent_output_fn + '/' + basename_fn + '/' + 's' + f"{stage_pos_fn:02}"
                    print('path_input_fn:'+str(path_input_fn))
                    print('path_pos_output_fn:'+str(path_pos_output_fn))
                    #check if file belongs to current stage_pos group


                    #rename file for correct stage position sort syntax (_sXX_)
                    print('file:'+file)
                            #Isolate the frame number (timepoint)
                    file_timepoint = file.split('_t',1)[-1].split('.', 1)[0]
                    print('file_timepoint:'+str(file_timepoint))
                    print('stage_pos_fn:'+str(stage_pos_fn))
                            # Rename file for correct time and stage position sort syntax (_sXX_ and _tXXX.)
                    file_newname = 's' + f"{stage_pos_fn:02}" +'_t'+f"{int(file_timepoint):03}"+'.TIF'
                    print('file_newname:'+file_newname)


                    #check if stage position directory already exists and move file into this folder if it does
                    if os.path.exists(path_pos_output_fn + '/' + image_type_fn+ '_images/'):
                        try:
                            print(path_pos_output_fn + '/' + image_type_fn+ '_images/')
                            shutil.move(path_input_fn + '/' + file, path_pos_output_fn + '/' + image_type_fn+ '_images/'+ file_newname)
                        except OSError as e:
                            # If it fails, inform the user.
                            print('Error: %s - %s.' % (e.filename, e.strerror))
                    #otherwise, create folder and move file into this folder. Also copy parent yaml file into this folder.
                    else:
                        try:
                            os.makedirs(path_pos_output_fn + '/' + image_type_fn+ '_images/')
                            shutil.copy(parent_output_fn + '/wc_dataset_'+image_type_fn+'.yaml', path_pos_output_fn + '/wc_dataset_'+image_type_fn+'.yaml')
                            shutil.move(path_input_fn + '/' + file, path_pos_output_fn + '/' + image_type_fn+ '_images/'+ file_newname)


                        except OSError as e:
                            # If it fails, inform the user.
                            print('Error: %s - %s.' % (e.filename, e.strerror))

path_input_fn:E:/Eclipse Data/20230919_NHCF-V_P5_Doxo/Sorted/tissue_ai
path_pos_output_fn:E:/Eclipse Data/20230919_NHCF-V_P5_Doxo/Sorted/tissue_ai/s10
file:tissue_ai_s10_t1.TIF
file_timepoint:1
stage_pos_fn:10
file_newname:s10_t001.TIF
path_input_fn:E:/Eclipse Data/20230919_NHCF-V_P5_Doxo/Sorted/tissue_ai
path_pos_output_fn:E:/Eclipse Data/20230919_NHCF-V_P5_Doxo/Sorted/tissue_ai/s10
file:tissue_ai_s10_t10.TIF
file_timepoint:10
stage_pos_fn:10
file_newname:s10_t010.TIF
E:/Eclipse Data/20230919_NHCF-V_P5_Doxo/Sorted/tissue_ai/s10/ph1_images/
path_input_fn:E:/Eclipse Data/20230919_NHCF-V_P5_Doxo/Sorted/tissue_ai
path_pos_output_fn:E:/Eclipse Data/20230919_NHCF-V_P5_Doxo/Sorted/tissue_ai/s10
file:tissue_ai_s10_t11.TIF
file_timepoint:11
stage_pos_fn:10
file_newname:s10_t011.TIF
E:/Eclipse Data/20230919_NHCF-V_P5_Doxo/Sorted/tissue_ai/s10/ph1_images/
path_input_fn:E:/Eclipse Data/20230919_NHCF-V_P5_Doxo/Sorted/tissue_ai
path_pos_output_fn:E:/Eclipse Data/20230919_NHCF-V_P5_Doxo/Sorted/tissu

## RUN CODE ##

**Section 1: Navigate to data folder containing .nd file**

In [57]:
print("Current config:")
print("Image Type:", image_type)
print("Fluorescent Images:", is_fl)
print("Track Images:", is_track)

#prompt user for the file directory. Will open as a popup window named "tk"
tk_root = tkinter.Tk()
print("Please open the directory that contains your .nd file")
path_input = askdirectory(title='Select Input Folder with .nd file')  # shows dialog box and return the path
print("Inputed path:", path_input)
tk_root.destroy()
path_output = path_input + '/Sorted'

if os.path.exists(path_output):
    path_out_new = input('Enter new output folder name')
    path_output = path_input + '/' + path_out_new
    os.makedirs(path_output)
#create a new  output directory
else:
    os.makedirs(path_output)

Current config:
Image Type: ph1
Fluorescent Images: False
Track Images: False
Please open the directory that contains your .nd file
Inputed path: E:/Eclipse Data/20230919_NHCF-V_P5_Doxo


**Section 2: Execute functions to prepare folder structure for woundcompute**

In [58]:
# Create yaml file for image type
create_yaml(path_output, image_type)
print("Created .yaml file")
# Create list of .nd files/experiments in the input folder
basename_list = define_basename_list(path_input)
print("Created basename list:", basename_list)

print("Sorting images into corresponding basename folders...")
# Sort images in input folder into Sorted/basename folder
sort_basename_folders(basename_list, path_input, path_output)
print("Completed!")

# Extract stage position information from .nd file
stage_pos_nd = [48, 48]
print("Extracted stage position information from .nd files")

print("Sorting images in each basename folder into their corresponding stage position folders...")
#  Sort images in each basename folder into their corresponding stage position folders
sort_stage_pos_folders(basename_list,path_output, stage_pos_nd, image_type)
print("Completed")


Created .yaml file
Created basename list: ['tissue_ai', 'tissue_bi']
Sorting images into corresponding basename folders...
Completed!
Extracted stage position information from .nd files
Sorting images in each basename folder into their corresponding stage position folders...


UnboundLocalError: local variable 'file_newname' referenced before assignment

In [49]:
stage_pos_nd = [48,48]
print("Extracted stage position information from .nd files")

print("Sorting images in each basename folder into their corresponding stage position folders...")
#  Sort images in each basename folder into their corresponding stage position folders
sort_stage_pos_folders(basename_list,path_output, stage_pos_nd, image_type)
print("Completed")


Extracted stage position information from .nd files
Sorting images in each basename folder into their corresponding stage position folders...


UnboundLocalError: local variable 'file_newname' referenced before assignment

**Section 3: Execute woundcompute for all basename folders in the Sorted folder**

In [ ]:
stage_pos_nd = [42]

print("Sorting images in each basename folder into their corresponding stage position folders...")
#  Sort images in each basename folder into their corresponding stage position folders
sort_stage_pos_folders(basename_list,path_output, stage_pos_nd, image_type)
print("Completed")


In [60]:
print("Starting wound compute for each stage position...")
for index, basename in enumerate(basename_list):
    time_all=[]
    for stage_pos in range(1, stage_pos_nd[index]+1):
        input_location = Path(path_output+'/'+basename+'/'+'s' + f"{stage_pos:02}")
        try:
            time_all, action_all = ia.run_all(input_location)
            print("tissue number:", stage_pos, "time:", time.time())
        except Exception as ex:
            time_all.append(time.time())
            print("tissue number:", stage_pos, "time:", time.time())
            print("---------ERROR OF SOME DESCRIPTION HAS HAPPENED-------")
            print(ex)
            print("------------------------------------------------------")

print("Completed! Hurray!")

print("Please run extract_dataset to consolidate outputs into an excel file!")

Starting wound compute for each stage position...


KeyboardInterrupt: 